# Make testing *way* easier by using tox

### About seamless CI, testing automation, best practices and cats 🐈

### Note: this is actually the second presentation of this cycle.
The underlying application (neon) has been used already during the first presentation, which was about mocking internal objects while writing (unit) tests. The notebook is publicly available over [github](https://github.com/jean-n92/triangle-community-presentations/blob/minor/refresh-and-refactor/notebooks/presentation-pytest.ipynb).

Neon is a simple application that, when invoked, calls the public API [catfacts](https://catfact.ninja/fact) and supplies the user with a certain number of interesting facts about cats. Here's an example:

In [ ]:
!cat-facts -f 5

![Tox logo](https://tox.wiki/en/rewrite/_static/tox.svg)

## What is tox?
#### *And (most importantly) how is it going to make my life any better?*

According to the [documentation](https://tox.wiki/en/rewrite/index.html):

**tox** is a generic virtual environment management and test command line tool you can use for:

- checking your package *builds* and *installs* correctly under *different environments*,

- *running your tests* in each of the environments with the test tool of choice,

- greatly *reducing boilerplate code* and merging agent-run and shell-based testing.

## Let's drill them down

### 1. Checking that your package *builds* and *installs* correctly under *different environments*

Suppose that I'd like to distribute my application into <mark>two different platforms</mark>: **Linux** and **Windows**. Moreover, I'd like my application to be compatible with **Python 3.6 to 3.9**. This forces me to make sure that both *my application* and *its dependencies* can indeed work into each one of these environments.    

Let's see a possible implementation of this CI requirement within Azure Pipeline (at the job level).

```yaml
jobs:
  - job:
    displayName: "Testing on"
    strategy:
      matrix:
```

```yaml
        py36-linux:
          displayName: "Python 3.6 (Linux)"
          pythonVersion: "3.6"
          imageName: "ubuntu-latest"
        py38-linux:
          displayName: "Python 3.8 (Linux)"
          pythonVersion: "3.8"
          imageName: "ubuntu-latest"
```

```yaml
        py37-win:
            displayName: "Python 3.7 (Windows)"
            pythonVersion: "3.7"
            imageName: "windows-latest"
        py39-win:
            displayName: "Python 3.9 (Windows)"
            pythonVersion: "3.9"
            imageName: "windows-latest"
```

I have now set up a deployment strategy that will work on all required environments/version. Let's proceed with running our tests at this point.

### 2. *Running your tests* in each of the environments with the test tool of choice

```yaml
    pool:
          vmImage: $(imageName)
    
    steps:
      - task: UsePythonVersion@0
        displayName: "Use Python $(pythonVersion)"
        inputs:
          versionSpec: "$(pythonVersion)"
```

```yaml
      - script: pip install --upgrade pip setuptools virtualenv
        displayName: "Upgrade existing pip"
```

```yaml
      - script: |
          python -m virtualenv venv
          source venv/bin/activate
          pip install --upgrade pip setuptools
          pip install -r requirements
```

```yaml
      - script: |
          python -m pytest -v -x -l \
            --cov neon/. \
            --cov-config $(Agent.BuildDirectory)/setup.cfg \
            --cov-report term-missing \
            tests/.
```

## Ok, great. Testing is done. Quite some code,  but I made it.

...but what if I now want to introduce a simple quality gate, such as `flake8`?

```yaml
      - script: pip install --upgrade flake8
        displayName: "Installing flake8"
```

```yaml
      - script: flake8 --config=$(Agent.BuildDirectory)/.flake8 neon/.
        displayName: "Running Flake8"
```

...but what if I now want to introduce another quality gate, such as `isort`?

![indecision](http://localhost:5500/.vscode/img4.jpg)

As you can see, this is already producing quite some code: in order to test my application, I have to download its requirements, create a virtual environment, initiate the environment, update its components, and finally run my preferred test tool. Moreover, it does not account for environment-specific requirements: let's say that, on `py36-linux`, I'd like to run my tests with a specific version of `urllib3` which is different than the standard one. How to do so?

That's where Tox comes into play.

### 3. Greatly *reducing boilerplate code* and merging agent-run and shell-based testing

```yaml
    pool:
          vmImage: $(imageName)
    
    steps:
      - task: UsePythonVersion@0
        displayName: "Use Python $(python.version)"
        inputs:
          versionSpec: "$(python.version)"
```

```yaml
      - script: pip install --upgrade pip tox
        displayName: "Install tox"
```

```yaml
      - script: tox
        displayName: "Run tox"
```

![whicone](http://localhost:5500/.vscode/img5.jfif)

And this is as much as I have to do! This step will simply run `tox` within my agent. Moreover, `tox` will automatically recognize the platform/version it's running onto, and will behave accordingly.

## How does our pipeline look like, at this point?

![pipeline](http://localhost:5500/.vscode/img2.png)

## How do I set it up?

Simply start by:
1. Installing tox via pip (`pip install tox`)
1. Creating a `tox.ini` configuration file inside your repo.

Setting tox up is very easy. Its configuration file follows the standard .ini structure, with sections and headers. The documentation about its [configuration](https://tox.wiki/en/rewrite/config.html) is quite extensive, and covers many possible use cases.

Let's have a look at its implementation for my cat application.

```ini
[tox]
envlist = py{36,37,38,39}-{lin,win}, flake8, isort
skip_missing_interpreters = true
```

```ini
[testenv]
platform =
    lin: linux
    win: win32

```

```ini
deps =
    pyspark~=3.1
    requests>=2.26
    urllib3==1.23
    pytest>=6.0.1
    pytest-cov~=3.0
```

```ini
commands_pre =
    pip install --upgrade pip setuptools
commands =
    python -m pytest --cov-config "{toxinidir}/tox.ini" tests/.

```

### Let's have our first test run!

In [ ]:
!tox -e py37

### Note: all environments inherit from `testenv`
This allows us to specify custom tools for custom testing scenarios

(remember the second promise: *running your tests in each of the environments <mark>with the test tool of choice</mark>*)

```ini
[testenv:py38-lin]
deps =
    pyspark~=3.1
    requests>=2.26
    urllib3==1.23
    pytest>=6.0.1
    coverage~=6.3
```

```ini
commands =
    coverage run -m pytest -c /dev/null tests/.
    coverage report
```

### Let's test it out!

In [ ]:
!tox -e py38-lin

### Note: I can even add options for other modules within `tox.ini`
This makes my life easier, since I do not have to rely on `.coveragerc`, `.flake8`, `pytest.ini` and all sort of additional sections within my `setup.cfg`.

```ini
[pytest]
addopts =
    -v -x -l
    --cov neon/.
    --cov-report term-missing
```

```ini
[coverage:report]
fail_under = 70
show_missing = True
exclude_lines =
    pragma: no cover
    def __repr__
    if __name__ == .__main__.:
```

### Note: quality gates can (and should!) be included in tox
The most common ones are **flake8** and **isort**. [Flake8](https://flake8.pycqa.org/en/latest/) is a well-known style-guide enforcer which makes sure that all code abides to PEP 8 guidelines. On the other hand, [isort](https://isort.readthedocs.io/en/latest/) takes care of my `import` statements: more often then not, applications end up having *a lot* of import statements, up to the point where it starts being hard making sense of them. As the name suggests, isort sorts them up for you in a PEP 8-approved order.

Both tools make your code way easier both to maintain for yourself and for others to contribute to, and are generally regarded as must-have for any respectable Python project in the real world.

```ini
[testenv:flake8]
deps =
    flake8>=4.0.0
commands =
    flake8 neon
```

```ini
[testenv:isort]
deps =
    isort~=5.10.0
commands =
    isort --check-only --diff neon/.
```

```yaml
      - script: tox -e flake8
        displayName: "Quality gate: Flake8"
```

```yaml
      - script: tox -e isort
        displayName: "Quality gate: isort"
```

### Let's test it out!

In [ ]:
!tox -e flake8 neon/.

## Is it reliable?

![spongebob](http://localhost:5500/.vscode/img3.jpg)

It has been adopted already by many major projects:
- [flask](https://github.com/pallets/flask/blob/master/tox.ini)    
- [scipy](https://github.com/scipy/scipy/blob/master/tox.ini)
- [requests](https://github.com/psf/requests/blob/master/tox.ini)    
- [numpy](https://github.com/numpy/numpy/blob/master/tox.ini)    

It is supported (and encouraged) by [Azure Pipelines](https://docs.microsoft.com/en-us/azure/devops/pipelines/ecosystems/python?view=azure-devops#run-tests-with-tox).

It has been starred more than 2.4k times on [GitHub](https://github.com/tox-dev/tox), and it is still [maintaned](https://github.com/tox-dev/tox/releases).

# Thank you for your attention 🙏🏻

### Useful links:
[Catfacts (API reference)](https://catfact.ninja/)    
[Previous presentation (Notebook)](https://github.com/jean-n92/triangle-community-presentations/blob/minor/refresh-and-refactor/notebooks/presentation-pytest.ipynb)    
[This application (Git repo)](https://github.com/jean-n92/triangle-community-presentations)    
[tox (Docs)](https://tox.wiki/en/rewrite/index.html)
[Automating Build, Test and Release Workflows with tox (Video)](https://youtu.be/PrAyvH-tm8E)